In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone --depth 1 https://github.com/NguyenNhoTrung/NLP-project.git

Cloning into 'NLP-project'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 1), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), 863.87 KiB | 6.00 MiB/s, done.


In [3]:
import shutil
import os
import torch
os.chdir('/content/NLP-project')

# 1. RoBERTa

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.9 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, RobertaForQuestionAnswering
from metric import predict, compute_exact_match, compute_f1

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = RobertaForQuestionAnswering.from_pretrained('roberta-base').to(device)
checkpoint = torch.load('/content/gdrive/MyDrive/Colab Notebooks/NLP project - checkpoint/Roberta/roberta.pt')
model.load_state_dict(checkpoint['model_state_dict'])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

<All keys matched successfully>

In [8]:
from utils import process_val
val_texts, val_queries, val_answers = process_val()
list_f1_score = []
list_em_score = []
for i in range(len(val_texts)):
    prediction = predict(model, tokenizer, val_texts[i], val_queries[i])

    list_f1_score.append(compute_f1(prediction, val_answers[i]['text']))
    list_em_score.append(compute_exact_match(prediction, val_answers[i]['text']))
print("Number of samples", len(list_f1_score))
print("F1 score: ", sum(list_f1_score) / len(list_f1_score))
print("EM score: ", sum(list_em_score) / len(list_em_score))

Number of samples 20302
F1 score:  0.8018790432817017
EM score:  0.6549601024529603


# 2. ELECTRA

In [9]:
from utils import process_val
from transformers import ElectraForQuestionAnswering

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('google/electra-base-discriminator')
model = ElectraForQuestionAnswering.from_pretrained('google/electra-base-discriminator').to(device)
checkpoint = torch.load('/content/gdrive/MyDrive/Colab Notebooks/NLP project - checkpoint/ELECTRA/ELECTRA.pt')
model.load_state_dict(checkpoint['model_state_dict'])

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.we

<All keys matched successfully>

In [11]:
val_texts, val_queries, val_answers = process_val()
list_f1_score = []
list_em_score = []
for i in range(len(val_texts)):
    prediction = predict(model, tokenizer, val_texts[i], val_queries[i])

    list_f1_score.append(compute_f1(prediction, val_answers[i]['text']))
    list_em_score.append(compute_exact_match(prediction, val_answers[i]['text']))
print("Number of samples", len(list_f1_score))
print("F1 score: ", sum(list_f1_score) / len(list_f1_score))
print("EM score: ", sum(list_em_score) / len(list_em_score))

Number of samples 20302
F1 score:  0.7437820376236882
EM score:  0.588907496798345


# 3.BERT

In [12]:
from utils import process_val
from transformers import BertForQuestionAnswering

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased').to(device)
checkpoint = torch.load('/content/gdrive/MyDrive/Colab Notebooks/NLP project - checkpoint/Bert/bert.pt')
model.load_state_dict(checkpoint['model_state_dict'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_out

<All keys matched successfully>

In [14]:
val_texts, val_queries, val_answers = process_val()
list_f1_score = []
list_em_score = []
for i in range(len(val_texts)):
    prediction = predict(model, tokenizer, val_texts[i], val_queries[i])

    list_f1_score.append(compute_f1(prediction, val_answers[i]['text']))
    list_em_score.append(compute_exact_match(prediction, val_answers[i]['text']))
print("Number of samples", len(list_f1_score))
print("F1 score: ", sum(list_f1_score) / len(list_f1_score))
print("EM score: ", sum(list_em_score) / len(list_em_score))

Number of samples 20302
F1 score:  0.7151951142680568
EM score:  0.5594522707122451
